In [ ]:
import torch
import torch.nn as nn
from model import Transformer
from config import get_config, get_weights_file_path
from train import get_model, get_ds, greedy_decode
import altair as alt
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Define the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [3]:
config = get_config()
train_dataloader, val_dataloader, vocab_src, vocab_tgt = get_ds(config)
model = get_model(config, vocab_src.get_vocab_size(), vocab_tgt.get_vocab_size()).to(device)

# Load the pretrained weights
model_filename = get_weights_file_path(config, f"29")
state = torch.load(model_filename)
model.load_state_dict(state['model_state_dict'])

Max length of source sentence: 53
Max length of target sentence: 64


<All keys matched successfully>

In [4]:
def load_next_batch():
    # Load a sample batch from the validation set
    batch = next(iter(val_dataloader))
    encoder_input = batch["encoder_input"].to(device)
    encoder_mask = batch["encoder_mask"].to(device)
    decoder_input = batch["decoder_input"].to(device)
    decoder_mask = batch["decoder_mask"].to(device)

    encoder_input_tokens = [vocab_src.id_to_token(idx) for idx in encoder_input[0].cpu().numpy()]
    decoder_input_tokens = [vocab_tgt.id_to_token(idx) for idx in decoder_input[0].cpu().numpy()]

    # check that the batch size is 1
    assert encoder_input.size(
        0) == 1, "Batch size must be 1 for validation"

    model_out = greedy_decode(
        model, encoder_input, encoder_mask, vocab_src, vocab_tgt, config['seq_len'], device)
    
    return batch, encoder_input_tokens, decoder_input_tokens

In [5]:
def mtx2df(m, max_row, max_col, row_tokens, col_tokens):
    return pd.DataFrame(
        [
            (
                r,
                c,
                float(m[r, c]),
                "%.3d %s" % (r, row_tokens[r] if len(row_tokens) > r else "<blank>"),
                "%.3d %s" % (c, col_tokens[c] if len(col_tokens) > c else "<blank>"),
            )
            for r in range(m.shape[0])
            for c in range(m.shape[1])
            if r < max_row and c < max_col
        ],
        columns=["row", "column", "value", "row_token", "col_token"],
    )

def get_attn_map(attn_type: str, layer: int, head: int):
    if attn_type == "encoder":
        attn = model.encoder.layers[layer].attention.attention_scores
    elif attn_type == "decoder":
        attn = model.decoder.layers[layer].self_attention.attention_scores
    elif attn_type == "encoder-decoder":
        attn = model.decoder.layers[layer].cross_attention.attention_scores
    return attn[0, head].data

def attn_map(attn_type, layer, head, row_tokens, col_tokens, max_sentence_len):
    df = mtx2df(
        get_attn_map(attn_type, layer, head),
        max_sentence_len,
        max_sentence_len,
        row_tokens,
        col_tokens,
    )
    return (
        alt.Chart(data=df)
        .mark_rect()
        .encode(
            x=alt.X("col_token", axis=alt.Axis(title="")),
            y=alt.Y("row_token", axis=alt.Axis(title="")),
            color="value",
            tooltip=["row", "column", "value", "row_token", "col_token"],
        )
        #.title(f"Layer {layer} Head {head}")
        .properties(height=400, width=400, title=f"Layer {layer} Head {head}")
        .interactive()
    )

def get_all_attention_maps(attn_type: str, layers: list[int], heads: list[int], row_tokens: list, col_tokens, max_sentence_len: int):
    charts = []
    for layer in layers:
        rowCharts = []
        for head in heads:
            rowCharts.append(attn_map(attn_type, layer, head, row_tokens, col_tokens, max_sentence_len))
        charts.append(alt.hconcat(*rowCharts))
    return alt.vconcat(*charts)

In [ ]:
# def get_all_attention_maps(attn_type: str, layers: list[int], heads: list[int], row_tokens: list, col_tokens, max_sentence_len: int):
#     charts = []
#     with open(f'attention_details_{attn_type}.txt', 'w', encoding='utf-8') as f:
#         f.write(f"Detailed Attention Analysis for {attn_type}\n")
#         f.write("=" * 50 + "\n\n")
        
#         for layer in layers:
#             f.write(f"\nLayer {layer}\n")
#             f.write("=" * 30 + "\n")
#             rowCharts = []
            
#             for head in heads:
#                 f.write(f"\nHead {head}\n")
#                 f.write("-" * 20 + "\n")
#                 attn_matrix = get_attn_map(attn_type, layer, head)
#                 f.write("\nAttention Matrix:\n")
#                 f.write("Row Token -> Column Token: Score\n")
                
#                 for row in range(min(len(row_tokens), max_sentence_len)):
#                     for col in range(min(len(col_tokens), max_sentence_len)):
#                         score = attn_matrix[row, col].item()
#                         if score > 0.1:
#                             f.write(f"{row_tokens[row]:10} -> {col_tokens[col]:10}: {score:.4f}\n")
                
#                 f.write("\n" + "-"*50 + "\n")
#                 rowCharts.append(attn_map(attn_type, layer, head, row_tokens, col_tokens, max_sentence_len))
            
#             charts.append(alt.hconcat(*rowCharts))
    
#     return alt.vconcat(*charts)

In [ ]:
batch, encoder_input_tokens, decoder_input_tokens = load_next_batch()
print(f'Source: {batch["src_text"][0]}')
print(f'Target: {batch["tgt_text"][0]}')
sentence_len = encoder_input_tokens.index("[PAD]")

Source: The train was delayed for one hour on account of the typhoon
Target: chuyến tàu đã bị trì hoãn trong một giờ vì lý do bão


In [100]:
import altair as alt
from altair_saver import save

# Tạo biểu đồ attention cho encoder
chart = get_all_attention_maps(
    attn_type="encoder",
    layers=[0, 1, 2],
    heads=[0, 1, 2, 3, 4, 5, 6, 7],
    row_tokens=encoder_input_tokens,
    col_tokens=encoder_input_tokens,
    max_sentence_len=min(20, sentence_len)
)

chart.save('chart_encoder.html')


In [101]:
import altair as alt
from altair_saver import save
chart = get_all_attention_maps(
    attn_type="decoder",
    layers=[0, 1, 2],
    heads=[0, 1, 2, 3, 4, 5, 6, 7],
    row_tokens=decoder_input_tokens,
    col_tokens=decoder_input_tokens,
    max_sentence_len=min(20, sentence_len)
)

chart.save('chart_decoder.html')


In [110]:
import altair as alt
from altair_saver import save
chart = get_all_attention_maps(
    attn_type="encoder-decoder",
    layers=[0, 1, 2],
    heads=[0, 1, 2, 3, 4, 5, 6, 7],
    row_tokens=decoder_input_tokens,
    col_tokens=encoder_input_tokens,
    max_sentence_len=min(20, sentence_len)
)

chart.save('chart_encoder_decoder.html')

In [103]:
layers = [0, 1, 2]
heads = [0, 1, 2, 3, 4, 5, 6, 7]

In [104]:
get_all_attention_maps("encoder", layers, heads, encoder_input_tokens, encoder_input_tokens, min(20, sentence_len))

alt.VConcatChart(...)

In [105]:
get_all_attention_maps("decoder", layers, heads, decoder_input_tokens, decoder_input_tokens, min(20, sentence_len))

alt.VConcatChart(...)

In [109]:
get_all_attention_maps("encoder-decoder", layers, heads, decoder_input_tokens, encoder_input_tokens, min(20, sentence_len))

alt.VConcatChart(...)

### Encoder Self‑Attention Visualization


- Mọi head đều thể hiện “tự chú ý” mạnh vào chính token:  
  - Ví dụ “train” → “train”, “delayed” → “delayed”…  
  - Cho thấy encoder ưu tiên bảo toàn thông tin gốc, nhưng có thể làm giảm sự tương tác ngữ cảnh nếu quá mức.
- Các token chức năng như SOS/EOS, mạo từ, giới từ được xử lý:  
  - SOS/EOS: ít tương tác với từ khác, nhưng nhiều head khác liên kết mạnh với “typhoon” trước khi kết thúc…
  - Mạo từ–Danh từ: “The” ↔ “train” được nhiều head liên kết, giúp giữ cú pháp chặt chẽ.
- Encoder tự động nhóm các từ thành cụm ý nghĩa:  
  - Định lượng: “one” → “hour”, biểu diễn chính xác cụm “one hour”.  
  - Giới từ: “of” → “account”, rồi kết nối thành chuỗi “on account of the typhoon”…  
- Mỗi head “đảm nhận” một khía cạnh khác nhau:  
  - Có head chuyên self‑focus (giữ vững thông tin bản thân).  
  - Có head gom nhóm từ gần (cú pháp, cụm từ). 
- Tuy nhiên, vẫn tồn tại sự trùng lặp vai trò giữa một số head.

### Decoder Self‑Attention Visualization

#### Giống với Encoder

- Giống như encoder, hầu hết các head trong decoder đều tập trung mạnh vào token hiện tại (high self‑focus), thể hiện qua các giá trị chú ý (attention scores) cao dọc theo đường chéo của ma trận. Ví dụ, Head 1 Layer 0 cho “tàu” → “tàu” score = 0.8426, Head 4 Layer 0 cho “hoãn” → “hoãn” score = 0.9676…

- Các token chức năng như SOS (Start‑of‑Sentence) vẫn giữ vai trò “neo” trong nhiều head, với giá trị chú ý cao vào chính nó và thấp với token khác. 
- Decoder tái hiện khả năng nhóm từ theo cụm ý nghĩa:  
  - “một” → “trong” (Head 4 L0: 0.9536) và “giờ” → “một” (Head 4 L0: 1.0365) thể hiện cụm “trong một giờ”
- Mỗi head tiếp tục đảm nhận một vai trò riêng:  
  - Có head tự tập trung (self‑focus) giữ thông tin token gốc  
  - Có head gom nhóm từ gần (cú pháp, cụm từ)
  - Mặc dù vậy, vẫn có sự chồng chéo chức năng giữa một số head.

#### Khác biệt với Encoder

- Decoder self‑attention sử dụng causal mask để chỉ cho phép nhìn thấy token trước (không nhìn thấy token tương lai), đảm bảo quá trình sinh tiếp theo tuân thủ tính tự hồi quy (autoregressive). Điều này dẫn đến ma trận chú ý tam giác dưới, thay vì ma trận đầy như encoder.
- Do mask, decoder không có các kết nối với future tokens, khiến một số head ít đa dạng hơn về mối quan hệ ngữ cảnh so với encoder.
- Nhiều head giống Head 1 Layer 1, Head 0 Layer 2 tập trung mạnh vào SOS hoặc các token đầu, hỗ trợ ổn định mở đầu của câu.
- Một số head vẫn bắt được mối quan hệ nhân-quả: “vì” → “bị” thể hiện liên kết nhân-quả
- Trong khi encoder ưu tiên bảo toàn thông tin gốc, decoder phải cân bằng giữa duy trì thông tin trước và kết nối ngữ cảnh đủ để sinh ra token tiếp theo, thể hiện qua việc một số head giảm diagonal dominance để tăng khả năng dự đoán.

### Cross-Attention Visualization

Cross‑attention trong decoder đóng vai trò cầu nối giữa query (token đầu ra tiếng Việt) và key/value (token đầu vào tiếng Anh), giúp mô hình dịch đúng cả từ vựng, ngữ pháp và cụm từ. Dưới đây là tổng hợp nhận xét kèm ví dụ số liệu:

- Khớp danh từ chủ đạo
    - Nhiều head tập trung “chuyến” → “train” với score cao, đảm bảo danh từ chính không bị sai lệch (Layer 1, Head 0: 0.9431; Layer 1, Head 3: 0.9662)

- Xử lý trợ động từ & cấu trúc chủ‑vị
    - Head gắn “tàu”/“đã” (query) với “was” (key) để giữ đúng nhịp câu (Layer 1, Head 0: “tàu” → “was” = 1.0125; Layer 2, Head 0: “tàu” → “was” = 1.0945)

- Nhóm trạng thái hoãn (“delayed” ↔ “trì”/“hoãn”/“bị”)
    - Các head nhấn mạnh mối liên hệ giữa hành động hoãn và giới từ “for” ( Layer 0, Head 3: “trì” → “delayed” = 0.9967; “hoãn” → “delayed” = 1.0081; Layer 1, Head 5: “bị” → “delayed” = 1.0834)

- Dịch cụm thời gian “for one hour” → “trong một giờ”
    - Token “trong”, “một”, “giờ” được chia sẻ giữa nhiều head để dịch mượt (Layer 0, Head 3: “một” → “one” = 1.1085; “giờ” → “hour” = 1.0992; “trong” → “for” = 0.9903) 

- Nhận biết điểm kết thúc câu (EOS)
    - Một số head liên kết token cuối (“bão”) với [EOS], giúp decoder hiểu khi nào dừng (Layer 1, Head 4: “bão” → [EOS] = 0.9469; Layer 2, Head 7: “bão” → [EOS] = 0.9973)